<a href="https://colab.research.google.com/github/PHaasbroek/Google-Colab/blob/main/Fluid-Solver/Fluid-Solver-Cpp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [67]:
# Header files need to be written at every compile to implement changes made to the header.

%%writefile functions.h

void printFunction();
double Re(double density, double dynamicViscosity, double charLength, double charSpeed);
double skinFriction_Schultz_Grunov(double Re);
double wallShearStress_f(double skinFriction, double density, double meanVelocity);


void printFunction(){
  std::cout << "Function Works 3" << std::endl;
}

double Re_f(double density, double dynamicViscosity, double charLength, double charSpeed){
  // density [kg/m^3]
  // dynamicViscosity [Pa * s] = [kg/m/s]
  // charLength [m]
  // charSpeed [m.s]

  return density * charSpeed * charLength / dynamicViscosity;
}


double skinFriction_Schultz_Grunov_f(double Re){
  // https://www.cfd-online.com/Wiki/Skin_friction_coefficient#References
  // Schultz-Grunov (equation 21.19a in Schlichting, Hermann (1979), Boundary Layer Theory, ISBN 0-07-055334-3, 7th Edition.)
  // skin friction on a flat plate for turbulent flow.
  // skinFriction [-]

  return 0.370 * pow(log10(Re),-2.584);
}

double wallShearStress_f(double skinFriction, double density, double meanVelocity){
  // wallShearStress [N/m^2]
  return 0.5 * skinFriction * density * meanVelocity * meanVelocity;
}




Overwriting functions.h


In [70]:
#Testing the Fluid-Solver-Cpp

%%writefile FluidSolverCpp.cpp

// Standard Libraries: ////////////////////////
#include <iostream>
#include <cmath>

// Custom headers files: /////////////////////////////
#include "functions.h"



int main(void) {

  // Input Parameters: ///////////////////////
  int cellNumberHeight = 100; // number of cells through the height of the domain
  double cellWidth = 1.0; // unit width of cells
  double firstCellHeight = 8e-5; // [m]
  double diameter = 0.050; // [m]
  double meanVelocity = 2.0; // [m/s]
  double density = 1000.0; // [kg/m^3]
  double dynamicViscosity = 0.01; // [Pa * s]

  double Re = Re_f(density, dynamicViscosity, diameter, meanVelocity);
  double skinFriction = skinFriction_Schultz_Grunov_f(Re);
  double wallShearStress = wallShearStress_f(skinFriction, density, meanVelocity);

  std::cout << Re << "\n";
  std::cout << skinFriction << "\n";

  //double wallShearStress = 0.5 * skinFriction * density * meanVelocity**2
  //double frictionVelocity = math.sqrt(wallShearStress / density)
  //double dimensionlessFirstCellHeight = density * frictionVelocity * firstCellHeight / dynamicViscosity #

/*
print("Reynolds number: " + str(Re) + " [-]")
print("Skin friction: " + str(skinFriction) + " [-]")
print("Wall shear stress: " + str(wallShearStress) + " [Pa]")
print("Friction velocity: " + str(frictionVelocity) + " [m/s]")
print("Dimensionless first cell height: " + str(dimensionlessFirstCellHeight) + " [-]")
*/


  std::cout << "Hello World" << std::endl;
  return 0;
  }


Overwriting FluidSolverCpp.cpp


In [71]:
%%script bash
g++ FluidSolverCpp.cpp -o FluidSolverCpp
./FluidSolverCpp

10000
0.0102915
Hello World
